In [ ]:
from src.config.data_loader_config import DATA_LOADER_CONFIG, OPTIMIZER_CONFIG
from src.data_loader.loader import Dataloader
from src.model.model import Model, Models, LossFunctions
from src.trainer.predict import save_result
from utils.fix_seed import set_seed
import src.callback as callback
import pytorch_lightning as pl
import os

# Parameters 설정
batch_size = DATA_LOADER_CONFIG['batch_size']
shuffle = DATA_LOADER_CONFIG['shuffle']
learning_rate = OPTIMIZER_CONFIG['learning_rate']
max_epoch = OPTIMIZER_CONFIG['max_epoch']
num_workers: int = DATA_LOADER_CONFIG.get('num_workers', 4)  # num_workers 기본값 4로 설정

set_seed(0)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
os.environ["TOKENIZERS_PARALLELISM"] = "false"
num_workers: int = DATA_LOADER_CONFIG.get('num_workers', 4)

In [ ]:
# 모델 선언
# select Model -> roberta_base, roberta_small, roberta_large, electra_base, electra_base_v3
model_name = 'synatra7b'
model = Model(Models.model_dict[model_name], learning_rate, LossFunctions.hu_loss).half()

from transformers import AutoTokenizer, DataCollatorWithPadding

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(Models.model_dict[model_name])

# 패딩 토큰 설정
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 패딩을 적용하는 데이터 콜레이터 설정
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
# Dataloader 선언
dataloader = Dataloader(batch_size, shuffle, Models.model_dict[model_name], check_aug=False, num_workers=num_workers)

# callback 정의
epoch_print_callback = callback.EpochPrintCallback()
checkpoint_callback = callback.ModelCheckpoint(model_name=model_name)
early_stopping = callback.EarlyStopping()
lr_monitor = callback.LearningRateMonitor()

trainer = pl.Trainer(
    accelerator='auto',
    devices='auto',
    max_epochs=max_epoch,
    callbacks=[lr_monitor, epoch_print_callback, checkpoint_callback, early_stopping],
    precision='16-mixed',
    deterministic=True
)

In [ ]:
print(model.dtype)

# 학습


In [ ]:
trainer = pl.Trainer(accelerator='gpu', devices=1, max_epochs=max_epoch, callbacks=[lr_monitor, epoch_print_callback,checkpoint_callback, early_stopping])
trainer.fit(model=model, datamodule=dataloader)
import torch
torch.cuda.empty_cache()
tokenizer parallelism = false

In [ ]:
import torch
torch.cuda.empty_cache()

# 추론


In [ ]:
checkpoint_callback.best_model_path

In [ ]:
checkpoint_callback.best_model_path

In [ ]:
# 가장 좋은 모델 불러오기
best_model_path = checkpoint_callback.best_model_path
model = Model.load_from_checkpoint(best_model_path, loss_func=LossFunctions.hu_loss)
trainer.test(model=model, datamodule=dataloader)
# 추론
predictions = trainer.predict(model=model, datamodule=dataloader, precision = '16-mixed')

# 결과 저장
save_result(predictions, model_name, max_epoch)